# 【共享函数】有关交易日期及时间处理

[传送门](https://www.joinquant.com/view/community/detail/c9827c6126003147912f1b47967052d9?type=1)

有关交易日期的问题：
- 获取前n个交易日；
- 有关日期的应用；
- datetime的常用方法；
- 获取某日前或者后第几个交易日的方法；
- 获取去年的今天对应的日期
- 获取持仓标的从买入到现在经历了几个交易日
- 获取每月/每周的 (倒数) 第N个交易日

In [2]:
from jqdata import *
import pandas as pd

## 获取前或后n天交易日的方法，以及datetime常用方法

In [3]:
def get_before_after_trade_days(date, count, is_before=True):
    """
    date :查询日期
    count : 前后追朔的数量
    is_before : True , 前count个交易日  ; False ,后count个交易日

    返回 : 基于date的日期, 向前或者向后count个交易日的日期 ,一个datetime.date 对象
    """
    from jqdata import  get_trade_days
    import pandas as pd
    all_date = pd.Series(get_all_trade_days())
    if isinstance(date,str):
        all_date = all_date.astype(str)
    if isinstance(date,datetime.datetime):
        date = date.date()

    if is_before :
        return all_date[all_date< date].tail(count).values[0]
    else :
        return all_date[all_date>date].head(count).values[-1]



date = "2019-10-20"#datetime.date(2019,10,20)
count = 10
is_before = False    
get_before_after_trade_days(date, count, is_before)

'2019-11-01'

## 获取去年的今天对应的日期

In [6]:
def get_last_year_date(context):
    """
    获取去年的今天对应的日期
        例如：现在是2018-08-15，则去年的今天是2017-08-15
    """
    now = context.current_dt
    last_one_year = int(now.year) - 1
    now_date = now.strftime("%Y-%m-%d")[-6:]
    last_year_date =  str(last_one_year) + now_date
    print(last_year_date)
    return last_year_date

## 获取持仓标的从买入到现在经历了几个交易日

In [5]:
import datetime

def initialize(context):
    # 初始化买入（一般不这样买入，此处为了方便测试函数）
    order('000001.XSHE', 100)
    run_daily(trade_days, time='10:00')

def trade_days(context):
    current_date = context.current_dt.date()
    hold_security = [p for p in context.portfolio.positions]
    print('今天持仓股票为:{0}'.format(hold_security))
    # 判断是否达到持仓日
    for stock in hold_security:
        # 第一次买入时间
        init_time = context.portfolio.positions[stock].init_time
        transact_time = context.portfolio.positions[stock].transact_time
        # 有多少个交易日
        trade_day = len(get_trade_days(start_date=init_time, end_date=current_date))
        print("{0}的买入时间为{1}，最后交易时间为{2}，今天的日期为{3}, 买入至今{4}个交易日".format(stock, 
                init_time, transact_time, current_date, trade_day))

## 获取每月/每周的 (倒数) 第N个交易日

In [7]:
def get_trade_day_bar(unit='W' , n=1 ,start_date=None,end_date=None,count=None):
    """
    unit : 频率 , "W"代表周 或者 "M"代表月
    n : 第几个交易日 , 每月第一个交易日: n=1 , 每月最后一个交易日 : n= -1 ,每月倒数第三个交易日 : n= -3 , 如果不足abs(n)个交易日,则返回符合条件的最近的一个交易日 
    start_date : 开始时间
    end_date : 结束时间
    count : 返回数据条数 , 指定start_date及count时返回start_date向后count条数据 , 指定end_date及count时返回end_date向前count条数据

    返回的数据会在 get_all_trade_days 以内
    返回 : 一个dataframe ,具体列含义见源码, date列就是需要获取的交易日了(datetime格式) 
    """
    import pandas as pd 
    from jqdata import get_all_trade_days,get_trade_days 

    df = pd.DataFrame(pd.to_datetime(get_all_trade_days()),columns=['date'])
    week_stamp = 24*60*60*7
    day_stamp = 24*60*60
    df['timestamp'] = df.date.apply(lambda x: x.timestamp() - day_stamp*3  )  #基于1970-01-01偏移三天
    df['mkweek'] = df.timestamp // week_stamp     # 1970-01-01 至今的第几周, 直接取每年的第几周再换年时会有问题
#     df['weekday'] = (df.timestamp %  week_stamp)/day_stamp   #周几
#     df['monthday'] = df.date.apply(lambda x : x.day)  #日
    df['month'] = df.date.apply(lambda x : x.month)   #月
    df['year'] = df.date.apply(lambda x : x.year)     #年

    if unit == "W":
        group_list = ['mkweek']
    elif unit =="M":
        group_list = ["year","month"]
    else :
        raise ValueError ('只支持M参数为 "M"或"W" ')

    if not isinstance(n , int):
        raise ValueError ('n 参数应该是一个int')
    elif n >0 :
        res =  df.groupby(group_list,as_index=False).head(n).groupby(group_list,as_index=False).last()
    elif n< 0:
        res =  df.groupby(group_list,as_index=False).tail(-n).groupby(group_list,as_index=False).first()    
    else :
        raise ValueError ('n 参数错误 : n={}'.format(n))


    if start_date and end_date and count:
        raise ValueError ('start_date ,end_date ,count 必须三选二')

    elif start_date and count :
        return res[res.date>=start_date].head(count)
    elif end_date and count :
        return res[res.date< =end_date].tail(count)
    elif start_date and end_date:
        return res[(res.date< =end_date)&amp;(res.date>=start_date)]
    elif not  start_date and not end_date and not  count:
        return res
    else:
        raise ValueError ('start_date ,end_date ,count 必须三选二')


res = get_trade_day_bar("M",-5,count=10,start_date='2015-09-15')
res

SyntaxError: invalid syntax (<ipython-input-7-e526b2f48666>, line 48)